##ライブラリ準備

In [ ]:
# TensorFlow と tf.keras のインポート
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D

# ヘルパーライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageFilter
import random, math
import cv2

##データ取得

In [ ]:
#画像データフォルダまでのパス
dir_path = "/content/drive/MyDrive/知識科学自習B/画像データ"

#データグループの一覧
group_list = os.listdir(path = dir_path)

#データグループの決定
i = 2
group_number = i

#データグループのパス
group_path = dir_path + group_list[group_number]
#データグループのカテゴリーの一覧
category_list = os.listdir(path = group_path)

print(group_list)
print(group_list[group_number])
print(category_list)

In [ ]:
# 画像をアスペクト比を保ったままリサイズ、左上詰め、空白部分には黒
def resize_gray_img(img_gray,x):
  h,w = img_gray.shape

  width = height = x

  scale_w = width/w
  scale_h = height/h
  if(scale_w < 1.0 or scale_h < 1.0):
    if scale_w < scale_h:
      resize_gray_img = cv2.resize(img_gray, dsize = None, fk = scale_w, fy = scale_w)
    else:
      resize_gray_img = cv2.resize(img_gray, dsize = None, fk = scale_h, fy = scale_h)
  else:
    if scale_w < scale_h:
      resize_gray_img = cv2.resize(img_gray, dsize = None, fk = scale_w, fy = scale_w)
    else:
      resize_gray_img = cv2.resize(img_gray, dsize = None, fk = scale_h, fy = scale_h)

  dst_img = np.zeros((height, width), dtype = np.unit8)

  # dst_imgにresize_imgを合成
  top = 0
  left = 0
  h, w = resize_gray_img.shape
  dst_img[0:h, 0:w] = resize_gray_img

  return dst_img

In [ ]:
# 画像用データ配列
X = []
# ラベル用データ配列
Y = []

# 画像データごとにadd_sample()を呼び出し、X,Yの配列を返す関数
def make_sample(files):
  global X,Y
  X = []
  Y = []
  for cat,fname in files:
    add_sample(cat, fname)
  return np.array(x), np.array(Y)

# 渡された画像データを読み込んでXに格納し、また、画像データに対応するcategoriesのidxをYに格納する関数
def add_sample(cat,fname):
  img_path = group_path + "/" + category_list[cat] + "/" + fname
  img = cv2.imread(img_path)
  # グレースケールに変換する
  img_gray = cv2.cvtColer(img, cv2.COLOR_BGR2GRAY)

  # 大津の手法で2値化する
  ret, img_binary = cv2.threshold(img_gray, 0, 255, cv2.THRSH_BINARY_INV + cv2.THRESH_OTU)

  # 輪郭を抽出する
  contours, hierarchy = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPEOX_NONE)

  # 最大の輪郭を取り出す
  max_countour  = max(contours, key=lambda x: cv2.countourArea(x))

  # 輪郭を描画する
  mask = cv2.drawContours(np.zeros_like(img_binary), contours, -1, color = 255, thickness = -1)

  # アルファチャンネル追加
  dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # maskの値が0のピクセルは透過する
  dst[mask == 0] = 0

  dst = resize_gray_img(dst, 512)

  data = np.array(dst)
  data = data / 255.0
  X.append(data)
  Y.append(cat)

# 全データ格納用配列
allfiles_train = []
allfiles_test = []

# カテゴリ配列の各値と、それに対応するidxを認識し、全データをallfilesにまとめる
for idx, cat in enumerate(category_list):
  allfiles = []
  file_path = group_path + "/" + cat
  files = os.listdir(path = file_path)
  for i in range(len(files)):
    allfiles.append((idx,files[i]))
  random.shuffle(allfiles)
  th = 8
  for i in range(0, th):
    allfiles_train.append((idx,files[i]))
  for i in range(th, len(files)):
    allfiles_test.append((idx,files[i]))

In [ ]:
# シャッフル後、学習データと検証データに分ける
train_images, train_labels = make_sample(allfiles_train)
test_images, test_labels = make_sample(allfiles_test)
xy = (train_images, test_images, train_labels, test_labels)

In [ ]:
class_names = category_list
plt.figure(figsize = (10, 10))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images[i],cmap = plt.cm.binary)
  plt.xlabel(class_names[train_labels[i]])
plt.show

##モデルの構築

層の設定

In [ ]:
input_shape = (80,512,512)
model = Sequential([
    Conv1D(25,128,activation = "relu", input_shape = input_shape[1:]),
    MaxPooling1D(),
    Conv1D(125,32,activation = "relu")
    MaxPooling1D(),
    Flatten(),
    Dense(128,activation = "relu"),
    Dense(10,activation = "sigmoid")
])

モデルのコンパイル

In [ ]:
model.compile(optimizer = "adam",
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"])

## モデルの訓練

In [ ]:
model.fit(train_images, train_labels, batch_size = 80, epochs = 10)

In [ ]:
model.summary()

## 正解率の評価

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, batch_size = 30, verbose = 2)

print("\nTest accuracy:",test_acc)

## 予測する

In [ ]:
predictions = model.predict(test_images)

In [ ]:
pridictions[0]
np.argmax(pridictions[0])
test_labels[0]

グラフ化

In [ ]:
# イメージ描写の関数
def plot_image(i, predictions_array, true_label, img):
  predictions_array, trus_label, img = predictions_array[i], true_label[i],img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, camp = plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = "blue"
  else:
    color = "red"

  plt.xlabel("{}{:2.0f}% ({})".format(class_names[predicted_label],
                                      100 * np.max(predictions_array),
                                      class_names[true_label]),
                                      color = color)

# 評価部分のバーグラフ
def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticls([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color = "#777777")
  plt.ylim([0,1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color("red")
  thisplot[true_label].set_color("blue")

In [ ]:
# X個のテスト画像、予測されたラベル、正解ラベルを表示します
# 正しい予測は青で、間違った予測は赤で表示しています
num_rows = 10
num_cols = 3
num_images = num_rows * num_cols
plt.figure(figsize = (2 * 2 *num_cols, 2 * num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
  plot_value_array(i, predictions, test_labels)
plt.show